In [1]:
from random import *

In [2]:
def tryRand(prob = .5):
    if random() < prob:
        return True
    else:
        return False

In [3]:
rule = [[3,0]
       ,[5,1]]

In [4]:
from math import log, sqrt
MAX_PLAYERS = 14
class Player:
    free_id = 0
    
    def __init__(self, name = "_NULL", isRandom = False):
        self.isRandom = isRandom
        self.name = name
        self.scores = []
        self.scoresPersonal = [ [] for i in range(MAX_PLAYERS)]
        self.score = [0] * MAX_PLAYERS
        self.streak = [0] * MAX_PLAYERS
        self.globalStreak = 0
        self.id = Player.free_id
        Player.free_id += 1
                
    def act(self, plr):
        return 0;

    def update(self, plr, act):
        pass

    def getTotalScore(self):
        sum = 0
        for score in self.score:
            sum += score
        return sum

    def clear(self):
        for i in range(MAX_PLAYERS):
            self.score[i] = 0
            self.streak[i] = 0
        self.globalStreak = 0
    
    def reset(self):
        for id in range(MAX_PLAYERS):
            self.scoresPersonal[id].append(self.score[id])
        self.scores.append(self.getTotalScore())
        self.clear()
    
    def getAverageScore(self, id = -1):
        sum = 0
        if id == -1:
            scores = self.scores
        else:
            scores = self.scoresPersonal[id]

        for score in scores:
            sum += score
        return sum // len(scores)

    def getMedianeScore(self, id = -1):
        if id == -1:
            scores = self.scores
        else:
            scores = self.scoresPersonal[id]
        n = len(scores)
        scores.sort()
        return scores[n//2]

    def getModaScore(self, id = -1):
        n = len(self.scores)

        if id == -1:
            scores = self.scores
        else:
            scores = self.scoresPersonal[id]

            
        mn = 999999
        mx = 0

        for i in range(n):
            mn = min(mn, scores[i])
            mx = max(mx, scores[i])

        diff = mx-mn
        k = int(log(n, 2))+1
        step = diff/k
        groupCount = [0] * k
        groupSum = [0] * k
        for j in range(k):
            for i in range(n):
                if mn + step*j <= scores[i] <= mn + step*(j+1):
                    groupCount[j] += 1
                    groupSum[j] += scores[i]

        bstGr = 0
        for j in range(k):
            if groupCount[j] > groupCount[bstGr]:
                bstGr = j

        return groupSum[bstGr] // groupCount[bstGr]

    def getDispersiaScore(self, id = -1):
        n = len(self.scores)

        x_ = self.getAverageScore(id)

        if id == -1:
            scores = self.scores
        else:
            scores = self.scoresPersonal[id]

        
        sTemp = 0
        for i in range(n):
            sTemp += (scores[i]-x_)**2

        ss = sTemp / (n+1)
        s = sqrt(ss)
        return int(s)

In [5]:
class Player1(Player):

    def __init__(self, name = "1 Player"):
        super().__init__(name)

    def act(self, plr):
        return 1;


class Player0(Player):

    def __init__(self, name = "0 Player"):
        super().__init__(name)

    def act(self, plr):
        return 0;


class PlayerCopycat(Player):
    def __init__(self, name = "Copycat", isRandom = True):
        super().__init__(name, isRandom)
        self.memory = [0] * MAX_PLAYERS

    def act(self, plr):
        return self.memory[plr.id];

    def update(self, plr, act):
        self.memory[plr.id] = act

    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.memory[i] = 0

class PlayerNegative(Player):
    def __init__(self, name = "Negative"):
        super().__init__(name)
        self.memory = [1] * MAX_PLAYERS

    def act(self, plr):
        return 1 - self.memory[plr.id];

    def update(self, plr, act):
        self.memory[plr.id] = act
    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.memory[i] = 0


class PlayerMercy20(Player):
    def __init__(self, name = "Mercy 20"):
        super().__init__(name)
        self.step = [0] * MAX_PLAYERS

    def act(self, plr):
        if self.step[plr.id] % 20 == 0:
            return 1
        else:
            return 0

    def update(self, plr, act):
        self.step[plr.id] += 1
        
    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.step[i] = 0


class PlayerFlag(Player):
    def __init__(self, name = "Flag Player"):
        super().__init__(name)
        self.flag = [0] * MAX_PLAYERS

    def act(self, plr):
        return self.flag[plr.id]

    def update(self, plr, act):
        if act == 1:
            self.flag[plr.id] = 1
    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.flag[i] = 0

class PlayerAlt(Player):
    def __init__(self, name = "Alt Player"):
        super().__init__(name)

    def act(self, plr):
        if plr.score[self.id] > self.score[plr.id]:
            return 0
        else:
            return 1

In [6]:
class PlayerFiftyFifty(Player):
    def __init__(self, name = "Fifty-Fifty"):
        super().__init__(name, True)

    def act(self, plr):
        if tryRand(0.5):
            return 1
        else:
            return 0


class Player90Zero(Player):
    def __init__(self, name = "90% Zero"):
        super().__init__(name, True)

    def act(self, plr):
        if tryRand(0.9):
            return 0
        else:
            return 1


class PlayerCopycatMercy(PlayerCopycat):
    def __init__(self, name = "Copycat Mercy"):
        super().__init__(name, True)

    def act(self, plr):
        if tryRand(.25):
            return 0
        else:
            return self.memory[plr.id]


class PlayerCopycatBroken(PlayerCopycat):
    def __init__(self, name = "Copycat Broken"):
        super().__init__(name,True)

    def act(self, plr):
        if tryRand(.25):
            return 1 - self.memory[plr.id]
        else:
            return self.memory[plr.id]

            
class PlayerRandomStart(Player):
    def __init__(self, name = "Random Start"):
        super().__init__(name,True)
        self.step = [0] * MAX_PLAYERS
        self.startLen = [0] * MAX_PLAYERS
        for i in range(MAX_PLAYERS):
            self.startLen[i] = randint(1, 50)
            self.step[i] = 0
    
    def act(self, plr):
        if self.step[plr.id] < self.startLen[plr.id]:
            return 1
        else:
            return 0

    def update(self, plr, act):
        self.step[plr.id] += 1

    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.startLen[i] = randint(1, 50)
            self.step[i] = 0


class PlayerRandomStreak(Player):
    def __init__(self, name = "Random Streak"):
        super().__init__(name,True)
        self.mode = [0] * MAX_PLAYERS
        self.len = [0] * MAX_PLAYERS
        for i in range(MAX_PLAYERS):
            self.mode[i] = 0
            self.len[i] = randint(0, 20)
            while self.len[i] == 0:
                self.switchMode(i)
    
    def act(self, plr):
        return self.mode[plr.id]
        
    def update(self, plr, act):
        self.len[plr.id] -= 1
        while self.len[plr.id] == 0:
            self.switchMode(plr.id)

    def reset(self):
        super().reset()
        for i in range(MAX_PLAYERS):
            self.mode[i] = 0
            self.len[i] = randint(0, 20)
            while self.len[i] == 0:
                self.switchMode(i)
            
    def switchMode(self, id):
        self.mode[id] = 1 - self.mode[id]
        self.len[id] = randint(0, 20)

class PlayerAltRand(Player):
    def __init__(self, name = "Alt Random"):
        super().__init__(name,True)

    def act(self, plr):
        if tryRand(0.9):
            return 1
        else:
            return 0

In [7]:
def compete(plrA, plrB):
    actA = plrA.act(plrB)
    actB = plrB.act(plrA)

    plrA.score[plrB.id] += rule[actA][actB]
    if rule[actA][actB] == 5:
        plrA.streak[plrB.id]+=1
    else:
        plrA.streak[plrB.id] = 0
    plrA.globalStreak = max(plrA.streak[plrB.id], plrA.globalStreak)
    
    plrB.score[plrA.id] += rule[actB][actA]
    if rule[actB][actA] == 5:
        plrB.streak[plrA.id]+=1
    else:
        plrB.streak[plrA.id] = 0
    plrB.globalStreak = max(plrB.streak[plrA.id], plrB.globalStreak)
    
    plrA.update(plrB, actB)
    plrB.update(plrA, actA)
    

In [8]:
Player.free_id = 0
players = [
            Player1("Alex"),
            Player0("Bob"),
            PlayerCopycat("Clara"),
            PlayerNegative("Denis"),
            PlayerMercy20("Emma"),
            PlayerFlag("Frida"),
            PlayerAlt("George"),

            PlayerFiftyFifty("Hank"),
            Player90Zero("Ivan"),
            PlayerCopycatMercy("Jack"),
            PlayerCopycatBroken("Kevin"),
            PlayerRandomStart("Lucas"),
            PlayerRandomStreak("Max"),
            PlayerAltRand("Natan"),

          ]
plrCnt = len(players)

In [9]:
for exp in range(150):
    for round in range(200):
        for i in range(plrCnt):
            for j in range(i+1, plrCnt):
                #if exp > 0 and not players[i].isRandom and not players[j].isRandom:
                #    continue
                compete(players[i], players[j])
    
    for plr in players:
        
        plr.reset()
        


In [10]:
#print(f"Total expirements - {len(players[0].scores)}")
#for plr in players:
#    print(f"{plr.name+',':16} avg - {plr.getAverageScore():5},\
# med - {plr.getMedianeScore():4} moda - {plr.getModaScore():4}, dispersia - {plr.getDispersiaScore():3}");

In [11]:
print(f"{"Average":18}", end="")
for plr in players:
    print(f"{plr.name:>10}",end="")
print()
for plr in players:
    print(f"{plr.name:16}", end="")
    print(end=" [")
    for i in range(plrCnt):
        print(f"{plr.getAverageScore(i):10}", end="")
    print("]")

Average                 Alex       Bob     Clara     Denis      Emma     Frida    George      Hank      Ivan      Jack     Kevin     Lucas       Max     Natan
Alex             [         0      1000       204       996       960       204       200       599       917       402       399       897       615       279]
Bob              [         0         0       600         0       570       600         0       297       541       600       450       522       305        59]
Clara            [       199       600         0       450       590       600       199       448       577       600       452       549       419       257]
Denis            [         0       999       450         0       939         6         0       455       883       523       454       876       510        80]
Emma             [        10       620       590        40         0        14        10       311       559       597       449       543       321        70]
Frida            [       199       600   

In [12]:
print(f"{"Median":18}", end="")
for plr in players:
    print(f"{plr.name:>10}",end="")
print()
for plr in players:
    print(f"{plr.name:16}", end="")
    print(end=" [")
    for i in range(plrCnt):
        print(f"{plr.getMedianeScore(i):10}", end="")
    print("]")

Median                  Alex       Bob     Clara     Denis      Emma     Frida    George      Hank      Ivan      Jack     Kevin     Lucas       Max     Natan
Alex             [         0      1000       204       996       960       204       200       600       920       400       400       900       616       276]
Bob              [         0         0       600         0       570       600         0       297       543       600       450       519       312        60]
Clara            [       199       600         0       450       590       600       199       448       579       600       454       543       419       257]
Denis            [         1      1000       450         0       940         6         1       456       886       523       456       884       515        82]
Emma             [        10       620       590        40         0        14        10       313       561       596       451       546       320        71]
Frida            [       199       600   

In [13]:
print(f"{"Moda":18}", end="")
for plr in players:
    print(f"{plr.name:>10}",end="")
print()
for plr in players:
    print(f"{plr.name:16}", end="")
    print(end=" [")
    for i in range(plrCnt):
        print(f"{plr.getModaScore(i):10}", end="")
    print("]")

Moda                    Alex       Bob     Clara     Denis      Emma     Frida    George      Hank      Ivan      Jack     Kevin     Lucas       Max     Natan
Alex             [         0      1000       204       996       960       204       200       597       932       405       399       954       632       272]
Bob              [         0         0       600         0       570       600         0       297       540       600       459       588       310        57]
Clara            [       199       600         0       450       590       600       199       447       579       600       466       507       413       256]
Denis            [         1      1000       450         0       940         6         1       436       871       521       458       978       525        89]
Emma             [        10       620       590        40         0        14        10       296       558       596       456       531       324        65]
Frida            [       199       600   

In [14]:
print(f"{"Dispersia":18}", end="")
for plr in players:
    print(f"{plr.name:>10}",end="")
print()
for plr in players:
    print(f"{plr.name:16}", end="")
    print(end=" [")
    for i in range(plrCnt):
        print(f"{plr.getDispersiaScore(i):10}", end="")
    print("]")

Dispersia               Alex       Bob     Clara     Denis      Emma     Frida    George      Hank      Ivan      Jack     Kevin     Lucas       Max     Natan
Alex             [         0         0         0         0         0         0         0        30        17        24        24        56        51        17]
Bob              [         0         0         0         0         0         0         0        21        12         0        19        46        40        12]
Clara            [         0         0         0         0         0         0         0        16         5         0        25        31        23        11]
Denis            [         0         0         0         0         0         0         0        33        24        14        19        71        72        17]
Emma             [         0         0         0         0         0         0         0        21        12         3        18        42        42        13]
Frida            [         0         0   